In [1]:
import os
import numpy as np
import torchvision.transforms as transforms
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from PIL import Image
from sklearn.metrics import classification_report as report

In [2]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
train_transformer = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomResizedCrop((224), scale=(0.5, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize
])

val_transformer = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize
])

In [3]:

txt_train_COVID=r'COVID\trainCT_COVID.txt'
txt_train_NonCOVID=r'NonCOVID\trainCT_NonCOVID.txt'


txt_val_COVID=r'COVID\valCT_COVID.txt'
txt_val_NonCOVID=r'NonCOVID\valCT_NonCOVID.txt'


txt_test_COVID=r'COVID\testCT_COVID.txt'
txt_test_NonCOVID=r'NonCOVID\testCT_NonCOVID.txt'


In [4]:
data_COVID = r'C:\Users\PC\ML_Classification_Project\COVID-19-CT-Classification\Data\CT_COVID'
data_NonCOVID = r'C:\Users\PC\ML_Classification_Project\COVID-19-CT-Classification\Data\CT_NonCOVID'

with open(txt_train_COVID, 'r') as file:
    COVID_train_paths = [os.path.join(data_COVID, line) for line in file.read().splitlines()]
    COVID_train_labels = np.ones(len(COVID_train_paths))

with open(txt_train_NonCOVID, 'r') as file:
    NonCOVID_train_paths = [os.path.join(data_NonCOVID, line) for line in file.read().splitlines()]
    NonCOVID_train_labels = np.zeros(len(NonCOVID_train_paths))

with open(txt_val_COVID, 'r') as file:
    COVID_val_paths = [os.path.join(data_COVID, line) for line in file.read().splitlines()]
    COVID_val_labels = np.ones(len(COVID_val_paths))

with open(txt_val_NonCOVID, 'r') as file:
    NonCOVID_val_paths = [os.path.join(data_NonCOVID, line) for line in file.read().splitlines()]
    NonCOVID_val_labels = np.zeros(len(NonCOVID_val_paths))

with open(txt_test_COVID, 'r') as file:
    COVID_test_paths = [os.path.join(data_COVID, line) for line in file.read().splitlines()]
    COVID_test_labels = np.ones(len(COVID_test_paths))

with open(txt_test_NonCOVID, 'r') as file:
    NonCOVID_test_paths = [os.path.join(data_NonCOVID, line) for line in file.read().splitlines()]
    NonCOVID_test_labels = np.zeros(len(NonCOVID_test_paths))


In [6]:
# Load and preprocess the images
train_paths = np.concatenate((COVID_train_paths, NonCOVID_train_paths))
val_paths = np.concatenate((COVID_val_paths, NonCOVID_val_paths))
test_paths = np.concatenate((COVID_test_paths, NonCOVID_test_paths))

train_labels = np.concatenate((COVID_train_labels, NonCOVID_train_labels))
val_labels = np.concatenate((COVID_val_labels, NonCOVID_val_labels))
test_labels = np.concatenate((COVID_test_labels, NonCOVID_test_labels))

train_images = []
for path in train_paths:
    image = Image.open(path).convert("RGB")
    image = train_transformer(image)  # Apply transformations
    train_images.append(image)

# Resize images to a consistent size
resized_train_images = [transforms.Resize((224, 224))(image) for image in train_images]

X_train = np.array([np.array(image) for image in resized_train_images])
X_train = X_train.reshape(X_train.shape[0], -1)
y_train = train_labels

val_images = []
for path in val_paths:
    image = Image.open(path).convert("RGB")
    image = val_transformer(image)  # Apply transformations
    val_images.append(image)

# Resize images to a consistent size
resized_val_images = [transforms.Resize((224, 224))(image) for image in val_images]

X_val = np.array([np.array(image) for image in resized_val_images])
X_val = X_val.reshape(X_val.shape[0], -1)
y_val = val_labels

test_images = []
for path in test_paths:
    image = Image.open(path).convert("RGB")
    image = val_transformer(image)  # Apply transformations
    test_images.append(image)

# Resize images to a consistent size
resized_test_images = [transforms.Resize((224, 224))(image) for image in test_images]

X_test = np.array([np.array(image) for image in resized_test_images])
X_test = X_test.reshape(X_test.shape[0], -1)
y_test = test_labels

# Perform PCA dimensionality reduction
pca = PCA(n_components=325)  # Choose the desired number of components

X_train_pca = pca.fit_transform(X_train)
X_val_pca = pca.transform(X_val)
X_test_pca = pca.transform(X_test)

# Train the SVM classifier
svm_classifier = SVC()
svm_classifier.fit(X_train_pca, y_train)

# Evaluate on the validation set
y_val_pred = svm_classifier.predict(X_val_pca)
val_accuracy = accuracy_score(val_labels, y_val_pred)
classification_report_result_val = report(y_val, y_val_pred)

# Evaluate on the test set
y_test_pred = svm_classifier.predict(X_test_pca)
test_accuracy = accuracy_score(test_labels, y_test_pred)
classification_report_result_test = report(y_test, y_test_pred)


print("Validation Accuracy:", val_accuracy)
print("Test Accuracy:", test_accuracy)
print("Validation class report: ", classification_report_result_val)
print("Test class report: ", classification_report_result_test)

Validation Accuracy: 0.6271186440677966
Test Accuracy: 0.625615763546798
Validation class report:                precision    recall  f1-score   support

         0.0       0.59      0.76      0.67        58
         1.0       0.68      0.50      0.58        60

    accuracy                           0.63       118
   macro avg       0.64      0.63      0.62       118
weighted avg       0.64      0.63      0.62       118

Test class report:                precision    recall  f1-score   support

         0.0       0.61      0.78      0.68       105
         1.0       0.66      0.46      0.54        98

    accuracy                           0.63       203
   macro avg       0.63      0.62      0.61       203
weighted avg       0.63      0.63      0.62       203



In [27]:
# Load and preprocess the images
train_paths = np.concatenate((COVID_train_paths, NonCOVID_train_paths))
val_paths = np.concatenate((COVID_val_paths, NonCOVID_val_paths))
test_paths = np.concatenate((COVID_test_paths, NonCOVID_test_paths))

train_labels = np.concatenate((COVID_train_labels, NonCOVID_train_labels))
val_labels = np.concatenate((COVID_val_labels, NonCOVID_val_labels))
test_labels = np.concatenate((COVID_test_labels, NonCOVID_test_labels))

train_images = []
for path in train_paths:
    image = Image.open(path).convert("RGB")
    image = train_transformer(image)  # Apply transformations
    train_images.append(image)

# Resize images to a consistent size
resized_train_images = [transforms.Resize((224, 224))(image) for image in train_images]

X_train = np.array([np.array(image) for image in resized_train_images])
X_train = X_train.reshape(X_train.shape[0], -1)
y_train = train_labels

val_images = []
for path in val_paths:
    image = Image.open(path).convert("RGB")
    image = val_transformer(image)  # Apply transformations
    val_images.append(image)

# Resize images to a consistent size
resized_val_images = [transforms.Resize((224, 224))(image) for image in val_images]

X_val = np.array([np.array(image) for image in resized_val_images])
X_val = X_val.reshape(X_val.shape[0], -1)
y_val = val_labels

test_images = []
for path in test_paths:
    image = Image.open(path).convert("RGB")
    image = val_transformer(image)  # Apply transformations
    test_images.append(image)

# Resize images to a consistent size
resized_test_images = [transforms.Resize((224, 224))(image) for image in test_images]

X_test = np.array([np.array(image) for image in resized_test_images])
X_test = X_test.reshape(X_test.shape[0], -1)
y_test = test_labels


# Train the SVM classifier
svm_classifier = SVC()
svm_classifier.fit(X_train, y_train)

# Evaluate on the validation set
y_val_pred = svm_classifier.predict(X_val)
val_accuracy = accuracy_score(val_labels, y_val_pred)
classification_report_result_val = report(y_val, y_val_pred)

# Evaluate on the test set
y_test_pred = svm_classifier.predict(X_test)
test_accuracy = accuracy_score(test_labels, y_test_pred)
classification_report_result_test = report(y_test, y_test_pred)


print("Validation Accuracy:", val_accuracy)
print("Test Accuracy:", test_accuracy)
print("Validation class report: ", classification_report_result_val)
print("Test class report: ", classification_report_result_test)

Validation Accuracy: 0.6186440677966102
Test Accuracy: 0.6403940886699507
Validation class report:                precision    recall  f1-score   support

         0.0       0.59      0.76      0.66        58
         1.0       0.67      0.48      0.56        60

    accuracy                           0.62       118
   macro avg       0.63      0.62      0.61       118
weighted avg       0.63      0.62      0.61       118

Test class report:                precision    recall  f1-score   support

         0.0       0.63      0.74      0.68       105
         1.0       0.66      0.53      0.59        98

    accuracy                           0.64       203
   macro avg       0.64      0.64      0.63       203
weighted avg       0.64      0.64      0.64       203

